In [1]:
'''
This is the second notebook. This tries to remove the PCA. 
Reduce number of kernels to 100
'''

'\nThis is the second notebook. This tries to remove the PCA. \nReduce number of kernels to 100\n'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras import Sequential, Model
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
from data import dnn_input
from model.RandomKernelModel import RandomKernelModel
from data import classifier_input
from sklearn.linear_model import RidgeClassifierCV
from sklearn.decomposition import KernelPCA
from sklearn.externals import joblib
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Data generation

In [3]:
train_data = dnn_input.get_processed_frames(os.path.abspath('./data/videos/frames_train/'))
test_data = dnn_input.get_processed_frames(os.path.abspath('./data/videos/frames_test/'))


## Model

In [57]:
model = RandomKernelModel.build_model((240, 320, 6),(10, 4), kernel_count=50)
# model.save_weights('./checkpoints/saved_model_withoutpca')
# model.load_weights('./checkpoints/saved_model_withoutpca')

In [58]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 240, 320, 6) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 240, 320, 6) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 240, 320, 50) 30050       input_5[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 10, 4)]      0                                      

In [59]:
output_train = model.predict(train_data, batch_size=16)

In [60]:
output_test = model.predict(test_data, batch_size=16)

In [61]:
np.save('dnn_model_output_without_pca', output_test)

## Generate dataset for classification

In [62]:
X_train, y_train = classifier_input.get_processed_classifier_input(output_train, normalized=True)
X_train_unbalanced, y_train_unbalanced = classifier_input.get_processed_classifier_input(output_train, normalized=True, balanced=False)
X_test, y_test = classifier_input.get_processed_classifier_input(output_test, normalized=True)
X_test_proper, y_test_proper = classifier_input.get_processed_classifier_input(output_test, normalized=True, balanced=False)

In [63]:
X_test_proper = np.asarray(X_test_proper)
y_test_proper = np.asarray(y_test_proper)

In [64]:
a, b = np.unique(y_test_proper, return_counts=True)

In [65]:
print(a)
print(b)

[0 1]
[4140  460]


In [66]:
np.save('X_train', X_train)
np.save('y_train', y_train)
np.save('X_train_unbalanced', X_train_unbalanced)
np.save('y_train_unbalanced', y_train_unbalanced)
np.save('X_test', X_test)
np.save('y_test', y_test)
np.save('X_test_proper', X_test_proper)
np.save('y_test_proper', y_test_proper)

In [67]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_train_unbalanced = np.load('X_train_unbalanced.npy')
y_train_unbalanced = np.load('y_train_unbalanced.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')
X_test_proper = np.load('X_test_proper.npy')
y_test_proper = np.load('y_test_proper.npy')

In [68]:
X_train.shape

(8280, 100)

## Dtat normalization

In [69]:
min_max_scaler = MinMaxScaler(feature_range=(-.5,.5))
normalizer = Normalizer()

X_train_cp = min_max_scaler.fit_transform(X_train)
X_test_cp = min_max_scaler.transform(X_test)
X_test_proper_cp = min_max_scaler.transform(X_test_proper)
X_train_unbalanced_cp =min_max_scaler.transform(X_train_unbalanced)

X_train_cp = normalizer.fit_transform(X_train)
X_test_cp = normalizer.transform(X_test)
X_test_proper_cp = normalizer.transform(X_test_proper)
X_train_unbalanced_cp = normalizer.transform(X_train_unbalanced)

In [70]:
joblib.dump(normalizer, './checkpoints/normalizer.joblib.pkl', 9)

['./checkpoints/normalizer.joblib.pkl']

## Dimensionality reduction

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20, whiten = False, random_state = 2019)
# pca = KernelPCA(kernel="rbf",n_components=154, gamma=None, fit_inverse_transform=True, random_state = 2019, n_jobs=1)

X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.transform(X_test)
X_pca_test_proper = pca.transform(X_test_proper)
# X_pca_reconst = pca.inverse_transform(X_pca)

In [ ]:
joblib.dump(pca, './checkpoints/pca.joblib.pkl', 9)

In [ ]:
a,b = np.unique(y_train, return_counts=True)
print(a)
print(b)

In [ ]:
plt.figure(figsize=(12,12))

plt.scatter(X_pca_train[y_train==0, 0], X_pca_train[y_train==0, 1], color='orange', alpha=0.5,label='0')
plt.scatter(X_pca_train[y_train==1, 0], X_pca_train[y_train==1, 1], color='pink', alpha=0.5,label='1')
plt.title("PCA")
plt.ylabel('Les coordonnees de Y')
plt.xlabel('Les coordonnees de X')
plt.legend()
plt.show()

In [ ]:
X_train_cp.shape

## SVM classifier

In [18]:
from sklearn.svm import SVC

In [19]:
clf = SVC(gamma='auto')

In [20]:
clf.fit(X_train_cp, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [21]:
print(clf.score(X_test_cp,y_test))
y_predict_proper = clf.predict(X_test_proper_cp)
a,b = np.unique(y_predict_proper, return_counts=True)
print(a)
print(b)

0.5106280193236715
[0 1]
[1038 3562]


## Advanced SVM

In [ ]:
from sklearn.svm import SVC
# clf = SVC(gamma='auto', kernel='rbf', probability=True, class_weight='balanced', C=1200000)
clf = SVC(gamma='auto', kernel='rbf', probability=True, class_weight='balanced', C=120000000)

clf.fit(X_train_unbalanced_cp, y_train_unbalanced) 

In [ ]:
print(clf.score(X_test_cp,y_test))

In [ ]:
print(clf.score(X_test_proper_cp, y_test_proper))

In [53]:
y_predict_proper = clf.predict(X_test_proper_cp)
a, b = np.unique(y_predict_proper, return_counts=True)
print(a)
print(b)

[0 1]
[3455 1145]


## Ridge Classifier

In [ ]:
classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = True)

In [ ]:
classifier.fit(X_pca_train, y_train)

In [ ]:
classifier.score(X_pca_test, y_test)

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=0)
clf.fit(X_pca_train, y_train)

In [ ]:
clf.score(X_pca_test, y_test)

In [ ]:
print(clf.score(X_pca_test,y_test))
y = clf.predict(X_pca_test)
a, b = np.unique(y, return_counts=True)
print(a)
print(b)

In [ ]:
print(clf.score(X_pca_test_proper, y_test_proper))
y_predict_proper = clf.predict(X_pca_test_proper)

In [ ]:
a,b = np.unique(y_test_proper, return_counts=True)
print(a)
print(b)

In [ ]:
a,b = np.unique(y_predict_proper, return_counts=True)
print(a)
print(b)

## Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=200, max_depth=7, random_state=0)
clf.fit(X_train_cp, y_train)
clf.score(X_test_cp, y_test)

In [ ]:
print(clf.score(X_test_proper_cp, y_test_proper))
y_predict_proper = clf.predict(X_test_proper_cp)
a,b = np.unique(y_predict_proper, return_counts=True)
print(a)
print(b)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=250, max_depth=3, random_state=0)
clf.fit(X_train_cp, y_train)


In [ ]:
clf.score(X_test_proper_cp, y_test_proper)

## XGBoost Classifier

In [ ]:
model = XGBClassifier(n_estimators=500, max_depth=5)

In [ ]:
model.fit(X_pca_train, y_train)

In [ ]:
model.score(X_pca_test, y_test)

## LGBM Classifier

In [186]:
stats.describe(X_train_cp[:, 50])

IndexError: index 50 is out of bounds for axis 1 with size 20

In [ ]:
X_train_cp.shape

In [187]:
clf = LGBMClassifier(n_estimators=2000)
clf.fit(X_train_cp, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [188]:
print(clf.score(X_test_cp, y_test))


0.741183574879227


In [189]:
print(clf.score(X_test_proper_cp, y_test_proper))
y_predict_proper = clf.predict(X_test_proper_cp)
a,b = np.unique(y_predict_proper, return_counts=True)
print(a)
print(b)

0.9063043478260869
[0 1]
[4137  463]


In [ ]:
clf = LGBMClassifier(n_estimators=2000)
clf.fit(X_pca_train, y_train)
print(clf.score(X_pca_test, y_test))


## Ada boost classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_pca_train, y_train)
clf.score(X_pca_test, y_test)

In [ ]:
print(clf.score(X_pca_test_proper, y_test_proper))
y_predict_proper = clf.predict(X_pca_test_proper)
a,b = np.unique(y_predict_proper, return_counts=True)
print(a)
print(b)

## Evaluations

In [54]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score

data = confusion_matrix(y_test_proper, y_predict_proper)

In [55]:
data

array([[3388,  752],
       [  67,  393]])

In [56]:
precision, recall, _, _ = precision_recall_fscore_support(y_test_proper, y_predict_proper, average='binary')
f1 = f1_score(y_test_proper, y_predict_proper)
print(precision, recall, f1)
print(recall)
print(f1)
print('{:.3f}, {:.3f}, {:.3f}'.format(int(precision*1000)/1000, int(recall*1000)/1000, int(f1*1000)/1000))

0.34323144104803494 0.8543478260869565 0.48971962616822423
0.8543478260869565
0.48971962616822423
0.343, 0.854, 0.489


In [ ]:
df_cm = pd.DataFrame(data, columns=np.unique(y_test_proper), index = np.unique(y_test_proper))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16})

In [ ]:
joblib.dump(clf, './checkpoints/lgbmclassifier.joblib.pkl', compress=9)

In [ ]:
clf2 = joblib.load('./checkpoints/random_forest_classifier.joblib.pkl')